# Access phase

In [1]:
###################################
###################################
# EXTRACTION DES DATAS OPENSESAME #
###################################
###################################
import csv
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import cv2 
import pandas as pd

# note: in the codes, VD is used as an abbreviation of visual disengagement (= gaze aversion)

#Change path to directory
set_WD = os.getcwd() # get current work directory

idd={'1':'P01','2':'P02','3':'P03','4':'P04','5':'P05', \
     '6':'P06','7':'P07','9':'P09','10':'P10','11':'P11', \
     '12':'P12','13':'P13','14':'P14','15':'P15','16':'P16', \
     '17':'P17','18':'P18', '19': 'P19', '20': 'P20', '21': 'P21', '22': 'P22', '23':'P23', '24':'P24', '25':'P25', '26':'P26', '27':'P27', \
     '28': 'P28', '29':'P29', '30':'P30', '31':'P31', '32':'P32', '33':'P33'} #list of all the participants

wd_outpout=set_WD+"/data/processed_data/T1_access.csv"
wd_input=set_WD+"/data/raw_data/task"


def ouvrir_fichier() :
    f=open(wd_outpout, "a")
    L="Sujet,sujet_num,Bloc,audio,Question,reponse,tps_acces,clarte,tempau,effort \n"
    f.writelines(L)
    f.close()
def completer_fichier(suj,suj_num,bloc,aud,numero,rep,tps,clarte,tempau,effort) :
    f=open(wd_outpout, "a")
    FF=suj+','+str(suj_num)+','+str(bloc)+','+aud+','+numero+','+str(rep)+','+str(tps)+','+clarte+','+tempau+','+effort+'\n'
    f.writelines(FF)
    f.close()
        
ouvrir_fichier()
os.chdir(wd_input)
lst = os.listdir()
lst.sort()


for filename in lst : 
    num=1
    data = pd.read_csv(filename,sep=',')
    sujet=filename[8:-6]
    
    bloc=filename[-5:-4]
    audio=data['audio']
    if float(sujet)<7:
        reponse=data['response_fin_acces']
        time=data['response_time_fin_acces']
    elif float(sujet)>6:
        reponse=data['response_fin_acces_1']
        time=data['response_time_fin_acces_1']
    clarte=data['response_Eval_clarte']
    tempau=data['response_Eval_tempau']
    effort=data['response_Eval_effort']    
    for i in range(len(data)) :
        aud=audio[i]
        rep=reponse[i]
        tps=str(time[i])
        clart=str(clarte[i])
        temp=str(tempau[i])
        eff=str(effort[i])
        numero=str(num)
        completer_fichier(idd[sujet],sujet,bloc,aud,numero,rep,tps,clart,temp,eff)
        num=num+1
 

In [2]:
##############################
##############################
# EXTRACTION QUALITY DEBRIEF #
##############################
##############################

wd_outpout=set_WD+"/data/processed_data/quality.csv"
wd_input=set_WD+'/data/raw_data/richesse_debrief'

def ouvrir_fichier() :
    f=open(wd_outpout, "a")
    L="Sujet,Bloc,Question,Etape_1,Etape_2\n"
    f.writelines(L)
    f.close()
def completer_fichier(suj,bloc,numero,etape_1,etape_2) :
    f=open(wd_outpout, "a")
    FF=suj+','+bloc+','+numero+','+etape_1+','+etape_2+'\n'
    f.writelines(FF)
    f.close()
        
ouvrir_fichier()
os.chdir(wd_input)
lst = [x for x in os.listdir() if x[-3:]=="csv"]
lst.sort()

for filename in lst : 
    print ("reading the file :",filename)
    data = pd.read_csv(filename,sep=',')   
    num=data['Question']
    sujet=data['Sujet']
    blocc=data['Bloc']
    step_1=data['Etape 1 result']
    step_2=data['Etape 2 result']    
    for i in range(len(data)) :
        numero=str(num[i])
        suj=str(sujet[i])
        bloc=str(blocc[i])
        etape_2=str(step_2[i])
        etape_1=step_1[i]
        completer_fichier(suj,bloc,numero,etape_1,etape_2)
 

reading the file : Evaluation_souvenirs_autobio.csv


## Merge gaze aversions data and memories characteristics in the same dataframe
#### for access period

In [3]:
def convert_to_str(input_seq, seperator):
    final_str = seperator.join(input_seq)
    return final_str

In [4]:
#######################
# FUSION T1 & QUALITY #
#######################

file_1=set_WD+"/data/processed_data/output_1.csv"
a_1 = pd.read_csv(set_WD+"/data/processed_data/quality.csv")
b_1 = pd.read_csv(set_WD+"/data/processed_data/T1_access.csv")
merged = b_1.merge(a_1)#pd.concat([b_1, a_1])
                   

merged.to_csv(file_1, index=False)

print('file opened and saved inside')


f=open(set_WD+"/data/processed_data/T1_quality.csv", "a")

inp=csv.reader(open(file_1,"r"))
ind_ligne=0

for row in inp:  
    L=convert_to_str(row, ',') 
    L=L+'\n'
    f=open(set_WD+"/data/processed_data/T1_quality.csv", "a")
    f.writelines(L)
    f.close()

file opened and saved inside


In [7]:
############################
# FUSION E.T. & T1+QUALITY #
############################

file=set_WD+"/data/processed_data/output.csv"
a = pd.read_csv(set_WD+"/data/processed_data/ratios_access.csv")
b = pd.read_csv(set_WD+"/data/processed_data/T1_quality.csv")
merged = b.merge(a)
merged.to_csv(file, index=False)

f=open(set_WD+"/data/processed_data/data_T1_access.csv", "a")
inp=csv.reader(open(file,"r"))

ind_ligne=0
for row in inp:  
    if row[15]!='MISSING' and row[10]!='NoA':        
        L=convert_to_str(row, ',') 
        if ind_ligne==1:
            if float(row[12])==0:
                L=L+',0'
            elif float(row[12])>0:
                L=L+',1'            
        if ind_ligne==0:
            L=L+',VD_here'
            ind_ligne=1
        L=L+'\n'
        f=open(set_WD+"/data/processed_data/data_T1_access.csv", "a")
        f.writelines(L)
        f.close()



In [8]:
access_VD = pd.read_csv(set_WD + '\data\processed_data\data_T1_access.csv')
###############################
# CREATE ORIENTATION_LAST.CSV #
###############################
# this file 
df = pd.read_csv(set_WD+"/data/processed_data/orientation.csv")
dfn=df.loc[df['last']!=0]
dfn.to_csv(set_WD+"/data/processed_data/last.csv")

file_2=set_WD+"/data/processed_data/last.csv"
f=open(set_WD+"/data/processed_data/orientation_last.csv", "a")
inp=csv.reader(open(file_2,"r"))

#### Calculation of the direction of the gaze aversion based on the last coordinate on the screen

In [9]:
# SCREEN 768 vertically and 1024 horizontally
# the screen is divided in 9 equal sections
ind_ligne=0
for row in inp:  
    L=convert_to_str(row, ',') 
    if ind_ligne==1:
        direction="inattendu"
        if float(row[5])<(1024/3) and float(row[6])<(768/3) : # row[5] = coord_x (horizontal)
            direction="upleft" # upper left corner
        if float(row[5])<(1024/3) and (768/3)<float(row[6])<((768/3)*2) : # row[5] = coord_x (horizontal)
            direction="left" 
        if float(row[5])<(1024/3) and ((768/3)*2)<float(row[6])<768 : # row[5] = coord_x (horizontal)
            direction="downleft" # lower left corner
        
        if (1024/3)<float(row[5])<((1024/3)*2) and float(row[6])<(768/3) : # row[5] = coord_x (horizontal)
            direction="up" # up
        if (1024/3)<float(row[5])<((1024/3)*2) and (768/3)<float(row[6])<((768/3)*2) : # row[5] = coord_x (horizontal)
            direction="center" #center of the screen
        if (1024/3)<float(row[5])<((1024/3)*2) and ((768/3)*2)<float(row[6])<768 : # row[5] = coord_x (horizontal)
            direction="down" # down
        
        if ((1024/3)*2)<float(row[5])<1024 and float(row[6])<(768/3) : # row[5] = coord_x (horizontal)
            direction="upright" # upper right corner
        if ((1024/3)*2)<float(row[5])<1024 and (768/3)<float(row[6])<((768/3)*2) : # row[5] = coord_x (horizontal)
            direction="right" 
        if ((1024/3)*2)<float(row[5])<1024 and ((768/3)*2)<float(row[6])<768 : # row[5] = coord_x (horizontal)
            direction="downright" # lower right corner    
        
        L=L+','+direction+',0'
    if ind_ligne==0:
        L=L+',direction,changed'
        ind_ligne=1
    L=L+'\n'
    f=open(set_WD+"/data/processed_data/orientation_last.csv", "a")
    f.writelines(L)
    f.close()


In [10]:
# delete files used for intermediate processing, which are now useless
os.remove(set_WD+"/data/processed_data/orientation.csv")
#os.remove(set_WD+"/data/processed_data/last.csv")
os.remove(set_WD+"/data/processed_data/T1_access.csv")
os.remove(set_WD+"/data/processed_data/output_1.csv")
os.remove(set_WD+"/data/processed_data/output.csv")
os.remove(set_WD+"/data/processed_data/quality.csv")

# To count the VD during access which continue during elaboration

In [11]:
nb_of_VD_access_continue_elaboration = 0
for i in range(len(access_VD['Sujet'])):

    if round(access_VD['moy_duree_VD'][i] + access_VD['delai'][i]) >= round(access_VD['tps_acces'][i]) :

        nb_of_VD_access_continue_elaboration = nb_of_VD_access_continue_elaboration + 1
print("nb_of_VD_access_continue_elaboration =", nb_of_VD_access_continue_elaboration)

nb_of_VD_access_continue_elaboration = 260


In [12]:
trials_with_VD = 0
trials_with_VD_continuing_in_visu = 0
for i in range(len(access_VD['Sujet'])):
    if access_VD['nb_VD'][i] > 0:
        trials_with_VD = trials_with_VD + 1
        if access_VD['VD_continue_in_visu '][i] == ' yes':
            trials_with_VD_continuing_in_visu = trials_with_VD_continuing_in_visu + 1
        
print('nb trials with VD =',trials_with_VD)
print('nb trials with VD continuing in elaboration = ', trials_with_VD_continuing_in_visu)

per_VD_continuing_in_visu = (trials_with_VD_continuing_in_visu/trials_with_VD)*100
print('% of VD continuing in elaboration =', per_VD_continuing_in_visu)

nb trials with VD = 492
nb trials with VD continuing in elaboration =  316
% of VD continuing in elaboration = 64.22764227642277
